In [1]:
from agential.cog.agent.critic import CriticAgent
from langchain_community.tools.google_serper.tool import GoogleSerperRun
from langchain_community.utilities.google_serper import GoogleSerperAPIWrapper
from langchain_community.utilities.google_search import GoogleSearchAPIWrapper
from langchain_community.chat_models.openai import ChatOpenAI
from agential.cog.prompts.critic import (
    # Prompts.
    CRITIC_POT_INSTRUCTION_GSM8K,
    CRITIC_CRITIQUE_INSTRUCTION_GSM8K, 
    CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_GSM8K,
    # Examples.
    GSM8K_FEWSHOT_EXAMPLES_POT,
    GSM8K_FEWSHOT_EXAMPLES_CRITIC,
    GSM8K_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,
)


import warnings
warnings.filterwarnings("ignore")

import dotenv

dotenv.load_dotenv()

llm = ChatOpenAI()
search = GoogleSearchAPIWrapper()

In [ ]:
from agential.cog.functional.critic import _prompt_agent

question = "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?"

code = _prompt_agent(
    llm=llm, question=question, examples=GSM8K_FEWSHOT_EXAMPLES_POT, prompt=CRITIC_POT_INSTRUCTION_GSM8K
)

In [ ]:
from agential.cog.functional.critic import _prompt_critique
question = "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?"

code = """clips_sold_in_april = 48
clips_sold_in_may = clips_sold_in_april / 30
answer = clips_sold_in_april + clips_sold_in_may"""

critique = _prompt_critique(
    llm=llm,
    question=question,
    examples=GSM8K_FEWSHOT_EXAMPLES_CRITIC,
    answer=code,
    critique="",
    additional_keys={
        "execution_status": "Done",
        "code_answer": "64"
    },
    prompt=CRITIC_CRITIQUE_INSTRUCTION_GSM8K,
).split("Here's")[0]

In [ ]:
responses = [
    code,
    "The code provided is incorrect.\n\nThe issue lies in the calculation of the number of clips sold in May. It is stated that Natalia sold half as many clips in May compared to April, which means she sold 48/2 = 24 clips in May. The code incorrectly calculates the number of clips sold in May by dividing the number of clips sold in April by 30.\n\nHere's the corrected version of the code:\n\n```python\nclips_sold_in_april = 48\nclips_sold_in_may = clips_sold_in_april / 2  # Natalia sold half as many clips in May\nanswer = clips_sold_in_april + clips_sold_in_may\n```"
]

In [ ]:
better = _prompt_critique(
    llm=llm,
    question=question,
    examples=GSM8K_FEWSHOT_EXAMPLES_CRITIC,
    answer=code,
    critique=critique + "\n\n" + "Here's a better solution:\n```python\n",
    additional_keys={
        "execution_status": "Done",
        "code_answer": "64"
    },
    prompt=CRITIC_CRITIQUE_INSTRUCTION_GSM8K,
).split("```")[0]

In [ ]:
print(better)

In [ ]:
"""
         | with tool | without tool |
PoT      |    o      |      o       |
"""

In [2]:
agent = CriticAgent(llm=llm, mode="code_interpreter", search=search)

In [3]:
question = "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with 4933828. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"

import warnings
warnings.filterwarnings("ignore")

out = agent.generate(
    question=question,
    examples=GSM8K_FEWSHOT_EXAMPLES_POT,
    prompt=CRITIC_POT_INSTRUCTION_GSM8K,
    critique_examples=GSM8K_FEWSHOT_EXAMPLES_CRITIC,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_GSM8K,
    use_interpreter_tool=True
)

<AGENT PROMPT==========================================================================>
# Write Python Code to solve the following questions. Store your result as a variable named 'answer'.

Question: Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
# Python code, return answer
total_eggs = 16
eaten_eggs = 3
baked_eggs = 4
sold_eggs = total_eggs - eaten_eggs - baked_eggs
dollars_per_egg = 2
answer = sold_eggs * dollars_per_egg

Question: A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?
# Python code, return answer
bolts_of_blue_fiber = 2
bolts_of_white_fiber = num_of_blue_fiber / 2
answer = bolts_of_blue_fiber + bolts_of_white_fiber

Question: Josh decides to try flipping a house.  He buys a house for

In [6]:
len(out)

6

In [7]:
out[0]

{'code': 'total_eggs = 16\neaten_eggs = 3\nbaked_eggs = 4933828\nsold_eggs = total_eggs - eaten_eggs - baked_eggs\ndollars_per_egg = 2\nanswer = sold_eggs * dollars_per_egg',
 'critique': 'The code is incorrect.\n\nThe issue with the code is in calculating the `sold_eggs`. The calculation for `sold_eggs` should be based on the eggs that are left after Janet eats three for breakfast and bakes muffins for her friends. The code should be corrected as follows:\n\n```python\ntotal_eggs = 16\neaten_eggs = 3\nbaked_eggs = 4933828\nremaining_eggs = total_eggs - eaten_eggs - baked_eggs\ndollars_per_egg = 2\nanswer = remaining_eggs * dollars_per_egg\n```',
 'improved_code': 'total_eggs = 16\neaten_eggs = 3\nbaked_eggs = 4933828\nremaining_eggs = total_eggs - eaten_eggs - baked_eggs\ndollars_per_egg = 2\nincome_per_day = remaining_eggs * dollars_per_egg\n\nanswer = income_per_day\n'}

In [8]:
out[1]

{'code': 'total_eggs = 16\neaten_eggs = 3\nbaked_eggs = 4933828\nremaining_eggs = total_eggs - eaten_eggs - baked_eggs\ndollars_per_egg = 2\nincome_per_day = remaining_eggs * dollars_per_egg\n\nanswer = income_per_day\n',
 'critique': 'The code is incorrect.\n\nThe issue in the code is the calculation of the remaining eggs. Since Janet eats 3 eggs for breakfast every morning and bakes 4933828 eggs into muffins, the calculation should be `remaining_eggs = total_eggs - eaten_eggs - baked_eggs`.\n\n',
 'improved_code': 'total_eggs = 16\neaten_eggs = 3\nbaked_eggs = 4933828\nremaining_eggs = total_eggs - eaten_eggs - baked_eggs\ndollars_per_egg = 2\nincome_per_day = remaining_eggs * dollars_per_egg\n\nanswer = income_per_day'}

In [9]:
out[-1]

{'code': "# Given values\ntotal_eggs_per_day = 16\neggs_eaten_for_breakfast = 3\neggs_used_for_muffins = 4933828\nprice_per_egg = 2\n\n# Calculate the remaining eggs available for sale\nremaining_eggs = total_eggs_per_day - eggs_eaten_for_breakfast - eggs_used_for_muffins\nif remaining_eggs < 0:\n    remaining_eggs = 0  # Ensure that the remaining eggs cannot be negative\n\n# Calculate the income Janet makes every day at the farmers' market\nincome_per_day = remaining_eggs * price_per_egg\n\nanswer = income_per_day",
 'critique': 'It is correct.'}